# Filtrado en el dominio de la frecuencia - parte 1

<div class="alert alert-block alert-success">
<b>Resumen:</b> Este notebook explora el uso de la Transformada de Fourier para analizar y filtrar imágenes en el dominio de la frecuencia utilizando OpenCV y Python. Se crean imágenes sintéticas, se calcula y visualiza la transformada de Fourier (magnitud y fase) tanto en 2D como en 3D, y se discute el desempeño de diferentes métodos.
</div>

***

## 1. Teoría de la Transformada de Fourier

### 1.1. Introducción a la Transformada de Fourier
La Transformada de Fourier es una herramienta matemática que descompone una señal (en este caso, una imagen) en sus componentes de frecuencia. En el contexto de imágenes, permite analizar cómo se distribuyen las diferentes frecuencias espaciales que componen la imagen.

- <b>Dominio Espacial:</b> Representación de la imagen en términos de píxeles.
- <b>Dominio de Frecuencia:</b> Representación de la imagen en términos de frecuencias espaciales, donde cada frecuencia indica patrones repetitivos como bordes y texturas.

### 1.2. Magnitud y Fase

- <b>Magnitud del Espectro:</b> Indica la amplitud de cada componente de frecuencia presente en la imagen. Las frecuencias bajas representan cambios suaves, mientras que las altas representan detalles finos y bordes.
- <b>Fase del Espectro:</b> Contiene información sobre la posición y alineación de las frecuencias. Es importante para la reconstrucción precisa de la imagen original.

---

Importación de librerias necesarias

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

## 2. Creación y visualización de una imagen sintética de prueba

Creamos una imagen sintética con un rectángulo blanco sobre un fondo negro para facilitar el análisis de la Transformada de Fourier.

In [ ]:
# Crear una imagen negra de 480x480 píxeles
img_box = np.zeros((480, 480), dtype=np.float32)

# Definir dimensiones del rectángulo
ancho = 20  # Ancho del rectángulo
alto = 50 # Alto del rectángulo

# Calcular el centro de la imagen
centro_y = img_box.shape[0] // 2
centro_x = img_box.shape[1] // 2

# Dibujar el rectángulo blanco en el centro
img_box[centro_y-alto//2:centro_y+alto//2, centro_x-ancho//2:centro_x+ancho//2] = 255

In [ ]:
# Visualización de la imagen sintética
plt.figure(figsize=(4,4))
plt.imshow(img_box, cmap='gray')
plt.title('Imagen sintética')
plt.axis('off')
plt.show()

## 3. Calculo de la transformada de Fourier (cv2.dft)

Utilizamos la función `cv2.dft` de OpenCV para calcular la Transformada de Fourier de la imagen.

In [ ]:
# Aplicar la Transformada de Fourier utilizando OpenCV
dft = cv2.dft(img_box, flags=cv2.DFT_COMPLEX_OUTPUT)

# Desplazar el espectro de Fourier para centrar las bajas frecuencias
dft_shift = np.fft.fftshift(dft)

### 3.1. Magnitud de la DFT

Calculamos la magnitud del espectro de Fourier para analizar las frecuencias presentes en la imagen.

In [ ]:
# Separar las partes real e imaginaria del espectro
real_part = dft_shift[:, :, 0]
imag_part = dft_shift[:, :, 1]

# Calcular la magnitud del espectro
magnitude_spectrum = cv2.magnitude(real_part, imag_part)

Normalización logarítmica, evitando log(0) y un resultado mayor o igual a cero

In [ ]:
# Normalizar la magnitud utilizando una escala logarítmica para mejorar la visualización
magnitude_spectrum_log = 20*np.log10(magnitude_spectrum + 1)  # Añadir 1 para evitar log(0)

# Normalizar la magnitud a un rango de 0 a 1
magnitude_spectrum_norm = cv2.normalize(magnitude_spectrum_log, None, 0, 1.0, cv2.NORM_MINMAX)

Visualizar la magnitud en 2d

In [ ]:
plt.figure()
plt.imshow(magnitude_spectrum_norm, cmap='gray')
plt.title('Transformada de Fourier - Magnitud con OpenCV')
plt.axis('off')
plt.show()

### 3.2. Visualización 3D del Espectro de Magnitud

Función para graficar la superficie 3d a partir de una matriz de dos dimensiones

In [ ]:
def plot3d( Z, title="Magnitud", elevation=45, azimute=35 ):
    """
    Función para graficar una superficie 3D a partir de una matriz 2D.
    
    Parámetros:
    - Z: Matriz 2D de datos.
    - title: Título del gráfico.
    - elevation: Ángulo de elevación de la vista.
    - azimute: Ángulo de azimut de la vista.
    """
    
    # Definir los valores de X e Y basados en el tamaño de la matriz Z
    x = np.arange(Z.shape[1])  # Valores del eje X
    y = np.arange(Z.shape[0])  # Valores del eje Y
    
    # Crear la rejilla de coordenadas
    X, Y = np.meshgrid(x, y)
    
    # Crear la figura y un subplot 3D
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    
    # Crear la superficie a partir de la matriz Z
    surf = ax.plot_surface(X, Y, Z, cmap='viridis')
    
    # Añadir una barra de color
    fig.colorbar(surf)
    
    # Añadir etiquetas a los ejes
    plt.title(f"{title} de la Transformada de Fourier")
    plt.xlabel("Frecuencia en el Eje X")
    plt.ylabel("Frecuencia en el Eje Y")
    
    # Ajustar manualmente la vista
    ax.view_init(elev=elevation, azim=azimute)  # Cambia estos valores para rotar el gráfico
    
    # Mostrar la gráfica
    plt.show()

Visualización 3d del espectro de magnitud

In [ ]:
plot3d(magnitude_spectrum_norm)

### 3.3. Fase de la DFT

La fase del espectro contiene información sobre la posición y estructura de los elementos en la imagen.

$\theta = $ `cv2.phase`( ... ) esta definida para $\theta = [0, \, 2\pi]$

In [ ]:
# Calcular la fase del espectro utilizando cv2.phase
phase = cv2.phase(real_part, imag_part)

# Normalizar la fase a un rango de 0 a 1 para visualización
phase_norm = cv2.normalize(phase, None, 0, 1.0, cv2.NORM_MINMAX)

Visualización de la fase en 2d

In [ ]:
plt.figure()
plt.imshow(phase_norm, cmap='gray')
plt.title('Transformada de Fourier - Fase con OpenCV')
plt.axis('off')
plt.show()

Visualización de la fase en 3d

In [ ]:
plot3d(phase_norm, title="Fase", elevation=65, azimute=10)

### 3.4. Guardando el espectro de amplitud y fase en un archivo binario

In [ ]:
# Guardar el espectro de magnitud
np.save('magnitude_spectrum.npy', magnitude_spectrum)

# Guardar la fase
np.save('phase_spectrum.npy', phase)

---

## Recursos adicionales

- https://docs.opencv.org/3.4/de/dbc/tutorial_py_fourier_transform.html